# Numerical Uncertainty Tutorial

## Theory

### Stochastic Arithmetic

Numerical analysis is the study of algorithms that use numerical approximation for problems involving mathematical analysis (Trefethen, 1992). Numerical analysis primarily involves studying numerical uncertainty, validating a program’s results, and the optimisation of operations. 

We focus on numerical uncertainty in order to get insight on the detection and quantification of numerical errors and their origin. The two methods of investigating numerical uncertainty are static and dynamic analysis. Static analysis analyses a program without executing it, thus it depends greatly on the algorithmic analysis of the program. While effective, static analysis cannot be scaled up to large code bases, which limits its scope. Dynamic analysis analyses a program during or after execution and while efficient, it is not generally as thorough as static analysis and is dependent on the program being analysed and its input data. Nonetheless, it is scalable and can be applied efficiently on large code bases.

Stochastic arithmetic, a form of dynamic analysis, analyses a program’s execution to measure numerical error by executing the program multiple times with random numerical perturbations.
Numerical uncertainty is determined by quantifying the errors that can occur in floating point arithmetic, these errors typically being rounding and truncation errors. 
More specifically, one type of rounding error, absorption errors, are caused by mathematical operations between large and small numbers where the shifting of decimal points in the mantissa causes a loss of precision. In turn, another rounding error, catastrophic cancellation, is the loss of leading significant digits caused by subtracting two approximately equal operands.
Two stochastic arithmetic methods that compute numerical uncertainty are DSA (Discrete Stochastic Arithmetic) (Vignes, 2004) and MCA (Monte Carlo Arithmetic) (Parker, 1997). Both techniques leverage randomness to model loss of accuracy inherent to floating point arithmetic due to its finite precision. 

<p align="center">
  <img src="./figures/numerical_analysis.png" alt="Image description" />
</p>


**MCA** is a variant of stochastic arithmetic that leverages randomness in fundamental floating-point operations, introducing a probabilistic element. We use it over DSA due to its scalability to large code bases. It simulates computations for a given virtual precision using the following perturbation: \

$$ inexact(x)=x+2^{e_{x}-t}\xi $$

where 
$$
e_{x} \textnormal{ is the exponent in the floating point representation of } x \\
        t \textnormal{ is the virtual precision and } \\
        \xi \textnormal{ is a random uniform variable of } (-\frac{1}{2},\frac{1}{2})
$$


MCA has three modes that allow perturbation to be introduced in the function input (Precision Bounding---PB), the function output (Random Rounding---RR) or both (full $mca\_mode$).  Full $mca\_mode$ combines RR and PB in the following equation: 

$$mca\_mode(x \circ y)=inexact_{RR}(inexact_{PB}(x) \circ inexact_{PB}(y))$$

Random Rounding tracks rounding errors, while Precision Bounding tracks catastrophic cancellations, and full $mca\_mode$ is a combination of both approaches.

**Verificarlo** (Denis et al., 2015) is a clang-based compiler that replaces floating point operations by a generic call to one of several configurable floating point models available.


<p align="center">
  <img src="./figures/verificarlo.png" alt="Image description" />
</p>

Verificarlo requires recompilation of a program in order to instrument the floating point operations, which is not always feasible on large, complex code bases such as Tensorflow or PyTorch.
This is especially difficult when the clang compiler used by Verificarlo is incompatible with the one used by the program or if the program calls upon many third-party tools which would also require recompilation. 
\
Nonetheless, MCA can be leveraged in programs through the use of the **Fuzzy ecosystem** (Kiar et al., 2020), a collection of Python software packages compiled with Verificarlo.
Fuzzy enables the instrumentation of different libraries by Verificarlo in order to quantify the numerical uncertainty present in code using these libraries.
The [Fuzzy ecosystem](https://github.com/verificarlo/fuzzy) with instrumented packages available such as the Python interpreter, the Numpy and SciPy packages and the libmath library.


<p align="center">
  <img src="./figures/tool_differences.png" alt="Image description" />
</p>


**Verrou** (Févotte et al., 2016) is a tool which also uses Monte Carlo Arithmetic to monitor the accuracy of floating point operations without needing to instrument the source code or recompile it. 
The Verrou tool is located at [Github](https://github.com/edf-hpc/verrou) and its documentation [here](https://edf-hpc.github.io/verrou/vr-manual.html#vr-cr.start-instrumentation).

Verrou is based upon Valgrind (Nethercote et al., 2007), a framework allowing for plug-in tools which use dynamic binary instrumentation (DBI), i.e. the process of modifying instructions of a binary program while it executes.


A comparison of the most popular implementations of MCA and DSA is shown in the table below:

|                          | **Verificarlo** | **Verrou** | **CADNA** |
|:-------------------------------:|:---------------:|:----------:|:---------:|
| **Theoretical Approach**        |      MCA        |     MCA    |    DSA    |
| **Practical Approach**          |    Compiler     |     DBI    | Manual Implementation |
| **Languages**                   |  C, C++, Fortran| All        | C, C++, Fortran |
| **Perturbation Type**           | RR, PB, full $mca\_mode$| RR, PB, full $mca\_mode$ | RR |
| **Parallelization**             | Yes             | No         | Conditionally upon Synchronisation |


**MENTION REPRODUCIBILITY CRISIS ?**

## Implementations

### Verificarlo

### Verrou

## Examples

## References
* L. N. Trefethen, “The definition of numerical analysis,” Cornell University, Tech. Rep., 1992.
* J. Vignes, “Discrete stochastic arithmetic for validating results of numerical software,” Numerical Algorithms, vol. 37, pp. 377–390, 2004.
* D. S. Parker, Monte Carlo Arithmetic: Exploiting Randomness in Floating-Point Arithmetic. University of California (Los Angeles). Computer Science Department, 1997.
* C. Denis, P. D. O. Castro, and E. Petit, “Verificarlo: Checking Floating Point Accuracy through Monte Carlo Arithmetic,” in 2016 IEEE 23nd Symposium on Computer Arithmetic (ARITH). Los Alamitos, CA, USA: IEEE Computer Society, jul 2016, pp. 55–62. [Online]. Available: https://doi.ieeecomputersociety.org/10.1109/ARITH.2016.31
* G. Kiar, P. de Oliveira Castro, P. Rioux, E. Petit, S. T. Brown, A. C. Evans, and T. Glatard, “Comparing Perturbation Models for Evaluating Stability of Neuroimaging Pipelines,” The International Journal of High Performance Computing Applications, vol. 34, no. 5, pp. 491–501, 2020.
* F. Févotte and B. Lathuilière, “Verrou: Assessing Floating-Point Accuracy Without Recompiling,” 2016.
* N. Nethercote and J. Seward, “Valgrind: A Framework for Heavyweight Dynamic Binary Instrumentation,” ACM Sigplan notices, vol. 42, no. 6, pp. 89–100, 2007.